<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/ML_final_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# Load the dataset
file_path = "/content/sample_data/SMCI.csv"
data = pd.read_csv(file_path)

# Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data.sort_values('Date', inplace=True)

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Feature Engineering: Including previous days' close prices as features
for lag in range(1, 8):
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

# Drop rows with NaN values created by the lag features
data.dropna(inplace=True)

# Define the feature set and target
X = data.drop(columns=['Close'])
y = data['Close']

# Standardize the feature set
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_cv_score = cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_cv_score = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Support Vector Machine
svm = SVR(kernel='rbf')
svm.fit(X_train, y_train)
svm_cv_score = cross_val_score(svm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')


In [3]:
# Prepare the last data point for prediction (using the latest available data)
last_data_point = data.loc[data.index[-1]]
last_data_point_scaled = scaler.transform([last_data_point.drop('Close').values])

# Function to predict future values
def predict_future_values(model, current_data, days):
    predictions = []
    for _ in range(days):
        next_pred = model.predict(current_data)
        predictions.append(next_pred[0])

        # Prepare the next data point (shift data and include new prediction)
        current_data = np.roll(current_data, -1)
        current_data[0, -1] = next_pred
    return predictions

# Predict the next 7 days
days_to_predict = 7
predictions_lr = predict_future_values(lr, last_data_point_scaled, days_to_predict)
predictions_rf = predict_future_values(rf, last_data_point_scaled, days_to_predict)
predictions_svm = predict_future_values(svm, last_data_point_scaled, days_to_predict)

# Generate the dates for the predictions
last_date = data.index[-1]
predicted_dates = [last_date + pd.Timedelta(days=i) for i in range(1, days_to_predict + 1)]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-3-648e2ad15a9c>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, -1] = next_pred
<ipython-input-3-648e2ad15a9c>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, -1] = next_pred
<ipython-input-3-648e2ad15a9c>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

In [4]:
# Function to predict previous values
def predict_previous_values(model, current_data, days):
    predictions = []
    for _ in range(days):
        next_pred = model.predict(current_data)
        predictions.append(next_pred[0])

        # Prepare the next data point (shift data and include new prediction)
        current_data = np.roll(current_data, 1)
        current_data[0, 0] = next_pred
    return predictions

# Predict the previous 7 days
days_to_predict = 7
prev_predictions_lr = predict_previous_values(lr, last_data_point_scaled, days_to_predict)
prev_predictions_rf = predict_previous_values(rf, last_data_point_scaled, days_to_predict)
prev_predictions_svm = predict_previous_values(svm, last_data_point_scaled, days_to_predict)

# Generate the dates for the previous predictions
first_date = train_end_date
prev_predicted_dates = [first_date - pd.Timedelta(days=i) for i in range(1, days_to_predict + 1)]
prev_predicted_dates.reverse()  # To maintain chronological order


<ipython-input-4-af509e89a661>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-4-af509e89a661>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-4-af509e89a661>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-4-af509e89a661>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error

NameError: name 'train_end_date' is not defined

In [5]:
# Function to predict previous values
def predict_previous_values(model, current_data, days):
    predictions = []
    for _ in range(days):
        next_pred = model.predict(current_data)
        predictions.append(next_pred[0])

        # Prepare the next data point (shift data and include new prediction)
        current_data = np.roll(current_data, 1)
        current_data[0, 0] = next_pred
    return predictions

# Predict the previous 7 days
days_to_predict = 7
prev_predictions_lr = predict_previous_values(lr, last_data_point_scaled, days_to_predict)
prev_predictions_rf = predict_previous_values(rf, last_data_point_scaled, days_to_predict)
prev_predictions_svm = predict_previous_values(svm, last_data_point_scaled, days_to_predict)

# Generate the dates for the previous predictions
# Get the date before the first date in the validation set
first_date = data.index[len(X_train)] - pd.Timedelta(days=1)
prev_predicted_dates = [first_date - pd.Timedelta(days=i) for i in range(1, days_to_predict + 1)]
prev_predicted_dates.reverse()  # To maintain chronological order

<ipython-input-5-75e8981a4ccf>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-5-75e8981a4ccf>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-5-75e8981a4ccf>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-5-75e8981a4ccf>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error

In [6]:
# Combine future and previous predictions into a single DataFrame
all_predicted_dates = prev_predicted_dates + predicted_dates
all_predictions_lr = prev_predictions_lr + predictions_lr
all_predictions_rf = prev_predictions_rf + predictions_rf
all_predictions_svm = prev_predictions_svm + predictions_svm

# Create a DataFrame to hold all predictions
all_predictions_df = pd.DataFrame({
    'Date': all_predicted_dates,
    'Linear Regression': all_predictions_lr,
    'Random Forest': all_predictions_rf,
    'SVM': all_predictions_svm
})

print(all_predictions_df)


         Date  Linear Regression  Random Forest         SVM
0  2023-06-29         846.580017     846.504609  253.799798
1  2023-06-30         862.108615     758.592293  195.947720
2  2023-07-01         834.882058     794.606086  195.947720
3  2023-07-02         846.241470     788.007297  195.947720
4  2023-07-03      205160.303423    1077.091090  195.947720
5  2023-07-04      208920.791841    1025.698589  195.947720
6  2023-07-05      202327.463081    1097.998981  195.947720
7  2024-07-06         846.580017     846.504609  253.799798
8  2024-07-07         388.584006     724.074890  195.947720
9  2024-07-08         849.923326     795.470695  195.947720
10 2024-07-09         842.987347     806.179391  195.947720
11 2024-07-10         820.779090     786.056387  195.947720
12 2024-07-11         830.552593     830.808180  195.947720
13 2024-07-12         905.442847     902.039979  195.947720
